In [2]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("[INST]What is the meaning of {word}[/INST]")

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={
        "max_new_tokens":250,
    }
)

chain = prompt | llm

chain.invoke({
    "word":"potato",
})

' Potato is a starchy vegetable that is grown underground in rows in a shallow, well-drained soil. It is a tuber, which means it is an underground, fleshy part of the plant that stores nutrients. Potatoes are a popular food source and are used in many different dishes, including salads, soups, stews, and baked dishes. They are also used as a source of starch for making paper and other products. Potatoes are high in carbohydrates and are a good source of fiber, vitamin C, and potassium.'